# Récupération des prix du carburant

In [8]:
import os.path
import zipfile
import requests
import xml.etree.ElementTree as ET

Les données du prix du carburant sont mises à disposition l'état français sur le site ["le prix des carburants"](https://www.prix-carburants.gouv.fr/rubrique/opendata/). Nous les récupérons sous forme d'une archive au format ZIP contenant les données au format XML. Nous récupérons le flux "instantané" (-10min).

In [3]:
data_url = "https://donnees.roulez-eco.fr/opendata/instantane"
data_rawResult = requests.get(data_url, allow_redirects=True)
data_zipFilename = "PrixCarburantsInstantane.zip"
data_xmlFilename = "PrixCarburants_instantane.xml"

Voici la description des données indiquée par le site:

| Nom | 	Type 	| Description |
|-----|-------------|-------------|
| pdv_liste | nœud  |	Liste des points de vente |
| pdv | nœud | Point de vente |
| id | attribut     | Identifiant du point de vente |
| latitude | attribut | Latitude (coordonnée géographique) à diviser par 100000 pour avoir la latitude en coordonnées GeoDecimal (WSG84) (*) |
| longitude | attribut | Longitude (coordonnée géographique) à diviser par 100000 pour avoir la longitude en coordonnées GeoDecimal (WSG84) (*) |
| cp | attribut | Code postal |
| pop |	attribut | Présence de la station (A = autoroute, R = route) |
| adresse |	nœud | Adresse du point de vente |
| ville | nœud | Ville |
| ouverture | nœud | Les horaires d’ouverture |
| debut | attribut | L’heure d’ouverture (hh:mm) |
| fin | attribut | L’heure de fermeture (hh:mm) |
| saufjour | attribut |	Jour(s) de fermeture (Lundi, Mardi, Mercredi, Jeudi, Vendredi, Samedi, Dimanche) |
| services | nœud |	Liste des services proposés par le point de vente |
| service | nœud | Nom du service |
| prix | nœud |	Prix du carburant |
| id | attribut | Id du carburant |
| nom |	attribut | Nom du carburant (Gazole, SP95, SP98, GPLc, E10, E85) |
| maj |	attribut | Date de la dernière mise à jour (AAAA-MM-JJ HH:mm:ss) |
| valeur | attribut | Prix en euros multiplié par 1000 |
| rupture |	nœud | Rupture de stock ou non-distribution d’un carburant |
| id | attribut | Id du carburant |
| nom |	attribut | Nom du carburant |
| debut | attribut | Date de début (AAAA-MM-JJ HH:mm:ss) |
| fin | attribut | Date de fin (si renseignée)  (AAAA-MM-JJ HH:mm:ss) |
| fermeture | nœud | fermeture du point de vente |
| type | attribut | Type de fermeture (temporaire ou définitive) |
| debut | attribut | Date de début (AAAA-MM-JJ HH:mm:ss) |
| fin | attribut |	Date de fin (si temporaire) (AAAA-MM-JJ HH:mm:ss) |

(*) : Les coordonnées du système du prix des carburants sont en PTV_GEODECIMAL. Cette unité est une variante du système standard geodecimal WGS84 (coordonnées GPS habituelles).

Pour convertir les coordonnées au format standard, il faut diviser les coordonnées fournies par 100000 :  
    
    Longitude standard  = Longitude fournie / 100000
    Latitude standard = Latitude fournie / 100000


Nous vérifions que le fichier existe en local avant de le télécharger:

In [4]:
dataAvail = False
if os.path.isfile(data_xmlFilename):
    dataAvail = True
else:
    dataAvail = False

Nous téléchargeons les données si elles n'existent pas encore en local, nous les extrayons de l'archive puis nous travaillons à partir de la version locale des données:

In [42]:
if dataAvail == False:
    open(data_zipFilename, 'wb').write(data_rawResult.content)
    with zipfile.ZipFile(data_rawFilename,"r") as zip_ref:
        zip_ref.extractall()
    os.remove(data_zipFilename)

tree = ET.parse(data_xmlFilename)
root = tree.getroot()

for prix in tree.findall(".//*[@cp='57580']/prix[@nom='Gazole']"):
    print(prix.attrib)

{'nom': 'Gazole', 'id': '1', 'maj': '2020-03-10 09:24:18', 'valeur': '1.424'}
{'nom': 'Gazole', 'id': '1', 'maj': '2020-04-09 06:35:26', 'valeur': '1.217'}
